In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"

In [2]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import dask
dask.__version__

'2022.03.0'

In [3]:
xgb.__version__

'1.6.1'

In [4]:
!nvidia-smi

Sun May 22 21:36:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    On   | 00000000:09:00.0 Off |                  N/A |
| 41%   44C    P8    30W / 280W |      5MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    On   | 00000000:41:00.0 Off |                  N/A |
| 41%   

In [5]:
!nvidia-smi -L

GPU 0: NVIDIA TITAN RTX (UUID: GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f)
GPU 1: NVIDIA TITAN RTX (UUID: GPU-50aeb092-88f5-4e0b-7f73-32741666f319)


In [6]:
train_x = dd.read_csv('../input/xgtrain.csv')
test_x = dd.read_csv('../input/xgval.csv')

In [7]:
train_y = train_x['target']
train_x = train_x[train_x.columns.difference(['target'])]

test_y = test_x['target']
test_x = test_x[test_x.columns.difference(['target'])]

In [8]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=["GPU-0eb32f58-b8d5-17c0-e952-f4ec26a9353f",
                                                 "GPU-50aeb092-88f5-4e0b-7f73-32741666f319"])
client = Client(cluster)


2022-05-22 21:36:56,743 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-05-22 21:36:56,753 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [9]:
dtrain = xgb.dask.DaskDMatrix(client, train_x, train_y)

In [10]:
dtest = xgb.dask.DaskDMatrix(client, test_x, test_y)

In [11]:
num_round = 1000

In [12]:
def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['multi:softprob']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3, 1.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3, 10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 20, 29),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'eval_metric': trial.suggest_categorical('eval_metric',['merror']),
        'num_class': 10,

    }

    output = xgb.dask.train(client, params, dtrain, num_round)
    
    booster = output['booster']  # booster is the trained model
    booster.set_param({'predictor': 'gpu_predictor'})

    predictions = xgb.dask.predict(client, booster, dtest)
    
    predictions = predictions.compute()
    
    predictions = np.argmax(predictions, axis=1)

    acc = accuracy_score(test_y, predictions)
    
    return acc

In [13]:
study = optuna.create_study(direction='maximize')

[I 2022-05-22 21:37:00,277] A new study created in memory with name: no-name-f2800484-e610-415a-b253-97884cc696a6


In [14]:
%%time
study.optimize(objective, n_trials=2)

[21:37:01] task [xgboost.dask]:tcp://127.0.0.1:40703 got new rank 0
[21:37:01] task [xgboost.dask]:tcp://127.0.0.1:35831 got new rank 1
[I 2022-05-23 02:42:50,281] Trial 0 finished with value: 0.9672619047619048 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.009918281416724807, 'alpha': 7.419020073614173, 'colsample_bytree': 0.921463217650149, 'subsample': 0.7867067475979554, 'learning_rate': 0.028327221135803687, 'max_depth': 27, 'min_child_weight': 4, 'eval_metric': 'merror'}. Best is trial 0 with value: 0.9672619047619048.
[02:42:50] task [xgboost.dask]:tcp://127.0.0.1:35831 got new rank 0
[02:42:50] task [xgboost.dask]:tcp://127.0.0.1:40703 got new rank 1
[I 2022-05-23 03:02:57,706] Trial 1 finished with value: 0.9723809523809523 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.007843886534117904, 'alpha': 0.01970718296905166, 'colsample_bytree': 0.8962444452978422, 'subsample': 0.976992663999547, 'learni

CPU times: user 8min 44s, sys: 3min 4s, total: 11min 48s
Wall time: 5h 25min 55s


In [15]:
%%time
study.optimize(objective, n_trials=50)

[03:02:57] task [xgboost.dask]:tcp://127.0.0.1:40703 got new rank 0
[03:02:57] task [xgboost.dask]:tcp://127.0.0.1:35831 got new rank 1
[I 2022-05-23 08:12:32,243] Trial 2 finished with value: 0.9660714285714286 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.36651945191994934, 'alpha': 3.3847526445059803, 'colsample_bytree': 0.7834250489411011, 'subsample': 0.7351438314508559, 'learning_rate': 0.01108018070539185, 'max_depth': 27, 'min_child_weight': 1, 'eval_metric': 'merror'}. Best is trial 1 with value: 0.9723809523809523.
[08:12:32] task [xgboost.dask]:tcp://127.0.0.1:35831 got new rank 0
[08:12:32] task [xgboost.dask]:tcp://127.0.0.1:40703 got new rank 1
[I 2022-05-23 18:21:33,242] Trial 3 finished with value: 0.9694047619047619 and parameters: {'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.12780107141273903, 'alpha': 4.637463020762252, 'colsample_bytree': 0.9383785662810683, 'subsample': 0.9261472426454624, 'learning

CPU times: user 1h 52min 16s, sys: 40min 18s, total: 2h 32min 35s
Wall time: 2d 20h 28min 23s


In [16]:
study.best_params

{'objective': 'multi:softprob',
 'tree_method': 'gpu_hist',
 'lambda': 0.0014345301923969806,
 'alpha': 0.4687380486697353,
 'colsample_bytree': 0.8434345456331338,
 'subsample': 0.8991048362147761,
 'learning_rate': 0.04635349901133093,
 'max_depth': 21,
 'min_child_weight': 4,
 'eval_metric': 'merror'}

In [17]:
study.best_value

0.9745238095238096

In [18]:
study.best_trial

FrozenTrial(number=46, values=[0.9745238095238096], datetime_start=datetime.datetime(2022, 5, 25, 22, 45, 49, 398971), datetime_complete=datetime.datetime(2022, 5, 25, 22, 52, 42, 128624), params={'objective': 'multi:softprob', 'tree_method': 'gpu_hist', 'lambda': 0.0014345301923969806, 'alpha': 0.4687380486697353, 'colsample_bytree': 0.8434345456331338, 'subsample': 0.8991048362147761, 'learning_rate': 0.04635349901133093, 'max_depth': 21, 'min_child_weight': 4, 'eval_metric': 'merror'}, distributions={'objective': CategoricalDistribution(choices=('multi:softprob',)), 'tree_method': CategoricalDistribution(choices=('gpu_hist',)), 'lambda': LogUniformDistribution(high=1.0, low=0.001), 'alpha': LogUniformDistribution(high=10.0, low=0.001), 'colsample_bytree': UniformDistribution(high=1.0, low=0.7), 'subsample': UniformDistribution(high=1.0, low=0.7), 'learning_rate': LogUniformDistribution(high=0.1, low=0.01), 'max_depth': IntUniformDistribution(high=29, low=20, step=1), 'min_child_weig